#### Valid WASM

In [3]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to JavaScript as byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binary string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if a start function is specified
     """))

In [4]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [5]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

Consider following WebAssembly programs. For each, argue if it is valid or not (i.e. produces an error when being loaded), if it traps at run-time, or if it works without error. In the last case, argue what it does. Use `!wat2wasm ...`, `runwasm(...)`, `runpywasm(...)`, and `runwasmer(...)` to check your answer! 

In [6]:
%%writefile a.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $program
    i32.const 3
    i32.const 4
    i32.add
    i32.add
  )
  (start $program)
)

Writing a.wat


YOUR ANSWER HERE

Instructor's Answer:

The second `i32.add` tries to pop two integers from the stack when there is only one. Program is rejected at compile-time due to type-checking.

In [7]:
!wat2wasm a.wat

a.wat:7:5: error: type mismatch in i32.add, expected [i32, i32] but got [i32]
    i32.add
    ^^^^^^^
a.wat:7:5: error: type mismatch at end of function, expected [] but got [i32]
    i32.add
    ^^^^^^^


In [8]:
%%writefile b.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (global $x (mut i32) i32.const 3)
  (global $y (mut i32) i32.const 5)
  (func $program
    global.get $x
    global.get $y
    i32.gt_s
    if
      global.get $x
      call $write
    else
      global.get $y
    end
  )
  (start $program)
)

Writing b.wat


YOUR ANSWER HERE

Instructor's Answer:

The else-branch would leave one more element on the stack than in the if-branch. Program is rejected at compile-time due to type-checking.

In [9]:
!wat2wasm b.wat

b.wat:14:5: error: type mismatch at end of `if false` branch, expected [] but got [i32]
    end
    ^^^


In [10]:
%%writefile c.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (global $x (mut i32) i32.const 3)
  (global $y (mut i32) i32.const 5)
  (func $program
    global.get $x
    global.get $y
    i32.gt_s
   ;; the size of the stacks that both branch end up with should be the same 
    if (result i32)
      global.get $x
    else
      global.get $y
    end
    call $write
  )
  (start $program)
)

Writing c.wat


YOUR ANSWER HERE

Instructor's Answer:

In this program, both the if-branch and the else-branch leave an integer on the stack, so it type-checks.

In [11]:
!wat2wasm c.wat

In [12]:
%%writefile d.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (global $x (mut i32) i32.const 5)
  (global $y (mut i32) i32.const 0)
  (func $program
    global.get $x
    global.get $y
    i32.div_s
    call $write
  )
  (start $program)
)

Writing d.wat


YOUR ANSWER HERE

Instructor's Answer:

Division by zero is a run-time error, not a compile-time error.

In [13]:
!wat2wasm d.wat
runwasm("d.wasm")

<IPython.core.display.Javascript object>

In [14]:
runpywasm("d.wasm")

Exception: pywasm: integer divide by zero

In [ ]:
runwasmer("d.wasm")

In [ ]:
%%writefile e.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $program
    i32.const 0
    i32.const 3
    i32.store offset=0
    i32.const 4
    i32.const 5
    i32.store offset=0
    i32.const 0
    i32.load offset=0
    i32.const 0
    i32.load offset=4
    i32.add
    call $write
  )
  (memory 1)
  (start $program)
)

YOUR ANSWER HERE

Instructor's Answer:

This program uses the memory to set, to get, and to add two variables. Unlike with local and global variables, memory locations don't have names but are accessed by computed addresses. 

In [15]:
!wat2wasm e.wat
runwasm("e.wasm")

e.wat: No such file or directory
unable to read file: e.wat


FileNotFoundError: [Errno 2] No such file or directory: 'e.wasm'

In [ ]:
runpywasm("e.wasm")

In [16]:
runwasmer("e.wasm")

FileNotFoundError: [Errno 2] No such file or directory: 'e.wasm'

In [17]:
%%writefile f.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $program
    i32.const 0
    i32.const 3
    i32.store offset=0
    i32.const 0
    i32.const 5
    i32.store offset=4
    i32.const -4
    i32.load offset=0
    i32.const 4
    i32.load offset=0
    i32.add
    call $write
  )
  (memory 1)
  (start $program)
)

Writing f.wat


YOUR ANSWER HERE

Instructor's Answer:

Memory is accessed with a computed address outside its boundary, causing a run-time error.

In [18]:
!wat2wasm f.wat
runwasm("f.wasm")

<IPython.core.display.Javascript object>

In [19]:
runpywasm("f.wasm")

Exception: pywasm: out of bounds memory access

In [20]:
runwasmer("f.wasm")

RuntimeError: RuntimeError: out of bounds memory access
    at <unnamed> (<module>[1]:0x44)

In [21]:
%%writefile g.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $max (param $x i32) (param $y i32) (result i32)
    (local $m i32)
    local.get $x
    local.get $y
    i32.gt_s
    if
      local.get $x
      local.set $m
    else
      local.get $y
      local.set $m
    end
    local.get $m
  )
  (func $program
    i32.const 3
    i32.const 5
    call $max
    call $write
  )
  (start $program)
)

Writing g.wat


YOUR ANSWER HERE

Instructor's Answer:

Two integer parameters are pushed on the stack before the call of `$max`, the call returns with one integer on the stack, which is then passed to `$write`, which removes the integer with the call.

In [22]:
!wat2wasm g.wat
runwasm("g.wasm")

<IPython.core.display.Javascript object>

In [23]:
runpywasm("g.wasm")

5 

In [24]:
runwasmer("g.wasm")

5 

In [25]:
%%writefile h.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $max (param $x i32) (param $y i32) (result i32)
    (local $m i32)
    local.get $x
    local.get $y
    i32.gt_s
    if
      local.get $x
      local.set $m
    else
      local.get $y
      local.set $m
    end
    local.get $m
  )
  (func $program
    i32.const 3
    call $max
    call $write
  )
  (start $program)
)

Writing h.wat


YOUR ANSWER HERE

Instructor's Answer:

Only one parameter is pushed on the stack before the call to `$max`, so type-checking reports an error.

In [26]:
!wat2wasm h.wat

h.wat:19:5: error: type mismatch in call, expected [i32, i32] but got [i32]
    call $max
    ^^^^


In [27]:
%%writefile i.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $max (param $x i32) (param $y i32) (result i32)
    (local $m i32)
    local.get $x
    local.get $y
    i32.gt_s
    if
      local.get $x
      local.set $m
    else
      local.get $y
      local.set $m
    end
  )
  (func $program
    i32.const 3
    i32.const 5
    call $max
    call $write
    i32.const 9
  )
  (start $program)
)

Writing i.wat


YOUR ANSWER HERE

Instructor's Answer:

In `$max`, the return value is missing, so type-checking fails. In `$program`, one extra value is pushed on the stack with `i32.const 9`, so type-checking fails as well.

In [28]:
!wat2wasm i.wat

i.wat:8:5: error: type mismatch in implicit return, expected [i32] but got []
    if
    ^^
i.wat:21:5: error: type mismatch at end of function, expected [] but got [i32]
    i32.const 9
    ^^^^^^^^^


In [29]:
%%writefile j.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $program
    (local $x i32)
    block
      i32.const 5
      local.set $x
      br 0
      i32.const 7
      local.set $x
    end
    local.get $x
    call $write
  )
  (start $program)
)

Writing j.wat


YOUR ANSWER HERE

Instructor's Answer:

After `$x` is set to `5`, the `br 0` instruction jumps to the end of the closest enclosing block, so `$x` is never set to `7` and `5` is printed.

In [30]:
!wat2wasm j.wat
runwasm("j.wasm")

<IPython.core.display.Javascript object>

In [31]:
runpywasm("j.wasm")

5 

In [32]:
runwasmer("j.wasm")

5 

In [33]:
%%writefile k.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $program
    (local $x i32)
    block
      i32.const 5
      local.set $x
      br 1
      i32.const 7
      local.set $x
    end
    local.get $x
    call $write
  )
  (start $program)
)

Overwriting k.wat


YOUR ANSWER HERE

Instructor's Answer:

The `br 1` instruction jumps to the end of the second most enclosing block, which is procedure `$program`, so `$x` is never printed.

In [34]:
!wat2wasm k.wat
runwasm("k.wasm")

<IPython.core.display.Javascript object>

In [35]:
runpywasm("k.wasm")

In [36]:
runwasmer("k.wasm")

In [ ]:
%%writefile l.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (func $program
    (local $x i32)
    block
      i32.const 5
      local.set $x
      br 2
      i32.const 7
      local.set $x
    end
    local.get $x
    call $write
  )
  (start $program)
)

YOUR ANSWER HERE

Instructor's Answer:

There is no enclosing block to which `br 2` could jump, to type-checking fails.

In [ ]:
!wat2wasm l.wat